In [ ]:
!pip3 install stardist

In [ ]:
!pip3 install ipywidgets

In [ ]:
!pip install cellpose

In [ ]:
!pip install cellseg_models_pytorch

In [22]:
# %% [markdown]
# # Project 2: Nuclei Segmentation with HoVer-Net (End-to-End)
#
# This notebook implements a robust, end-to-end pipeline using the **HoVer-Net** model. This approach is more powerful and reliable than a two-stage process.
#
# 1.  **Data Loading**: We create a PyTorch Dataset that loads images and their corresponding XML annotations.
# 2.  **Fine-Tuning**: A pre-trained HoVer-Net model is fine-tuned on the `train` set.
# 3.  **Validation**: The `val` set is used after each epoch to check performance and save the best model.
# 4.  **Inference**: The best model is used to generate predictions on the `test` set.

# %%
import os
import cv2
import torch
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from albumentations import Resize, Compose, Normalize
from albumentations.pytorch import ToTensorV2
import warnings

# --- Import HoVer-Net Model ---
from cellseg_models_pytorch.models import hovernet

# --- Suppress unnecessary warnings ---
warnings.filterwarnings("ignore", category=UserWarning)

# %% [markdown]
# ## 1. Configuration and Device Setup
#
# Here, we define all paths and hyperparameters. The code will automatically detect and use your Apple Silicon (MPS) GPU if available.

# %%
class CONFIG:
    # --- Paths (including the validation set) ---
    BASE_DIR = "kaggle-data/"
    TRAIN_IMG_DIR = os.path.join(BASE_DIR, "train")
    TRAIN_XML_DIR = os.path.join(BASE_DIR, "train")
    VAL_IMG_DIR = os.path.join(BASE_DIR, "val")
    VAL_XML_DIR = os.path.join(BASE_DIR, "val")
    TEST_IMG_DIR = os.path.join(BASE_DIR, "test_final")

    MODEL_SAVE_PATH = "hovernet_best_model.pth"

    # --- Model & Training Parameters ---
    IMG_SIZE = 256   # HoVer-Net is often trained on smaller patches
    BATCH_SIZE = 4   # Use a smaller batch size for this larger model
    EPOCHS = 30
    LR = 1e-4

    # --- Class Mapping (Important: 0 is background) ---
    CLASSES = ["Epirthelial", "Lymphocyte", "Macrophage", "Neutrophil"]
    CLASS_MAP = {name: i + 1 for i, name in enumerate(CLASSES)} # 1-based indexing for classes
    INV_CLASS_MAP = {i + 1: name for i, name in enumerate(CLASSES)}

# --- Setup Device (MPS for Apple Silicon, CPU fallback) ---
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("Using Apple MPS (GPU) 🚀")
else:
    DEVICE = torch.device("cpu")
    print("Using CPU")

# %% [markdown]
# ## 2. Data Loading and Helper Functions
#
# These utilities prepare the data in the specific format required for training the HoVer-Net model.

# %%
def rle_encode_instance_mask(mask: np.ndarray) -> str:
    """Encodes a 2D instance mask to a RLE triple string."""
    pixels = mask.flatten(order="F").astype(np.int32)
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    rle = []
    for i in range(len(runs) - 1):
        val, start, length = pixels[runs[i]], runs[i], runs[i+1] - runs[i]
        if val > 0:
            rle.extend([val, start, length])
    return " ".join(map(str, rle)) if rle else "0"

def create_training_maps(xml_path, shape):
    """Parses XML to create instance and type maps for training."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    inst_map = np.zeros(shape, dtype=np.int16)
    type_map = np.zeros(shape, dtype=np.int16)
    inst_id_counter = 1

    for ann_elem in root.findall('Annotation'):
        class_name = ann_elem.get('Name')
        if class_name in CONFIG.CLASS_MAP:
            class_id = CONFIG.CLASS_MAP[class_name]
            for region_elem in ann_elem.findall('.//Region'):
                vertices = [(float(v.get('X')), float(v.get('Y'))) for v in region_elem.findall('.//Vertex')]
                polygon = np.array(vertices, dtype=np.int32)
                cv2.fillPoly(inst_map, [polygon], inst_id_counter)
                cv2.fillPoly(type_map, [polygon], class_id)
                inst_id_counter += 1
    return inst_map, type_map

class NucleiDataset(Dataset):
    """Custom PyTorch Dataset for loading images and masks for HoVer-Net."""
    def __init__(self, image_dir, xml_dir, transform=None):
        self.image_dir = image_dir
        self.xml_dir = xml_dir
        self.file_ids = [os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith(".tif")]
        self.transform = transform

    def __len__(self):
        return len(self.file_ids)

    def __getitem__(self, idx):
        image_id = self.file_ids[idx]
        img_path = os.path.join(self.image_dir, f"{image_id}.tif")
        xml_path = os.path.join(self.xml_dir, f"{image_id}.xml")

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, _ = image.shape
        inst_map, type_map = create_training_maps(xml_path, (h, w))

        if self.transform:
            transformed = self.transform(image=image, masks=[inst_map, type_map])
            image = transformed["image"]
            inst_map, type_map = transformed["masks"]

        # The model expects a stacked label map
        labels = np.stack([inst_map, type_map], axis=0)
        return image, torch.from_numpy(labels).long()

# %% [markdown]
# ## 3. Model Training and Validation
#
# We fine-tune a pre-trained HoVer-Net model. After each epoch, we evaluate on the validation set and save the model if its performance improves.

# %%
def train_and_validate():
    print("--- Initializing Model and DataLoaders ---")
    
    # --- CORRECTED Model Initialization ---
    num_classes = len(CONFIG.CLASSES) + 1  # Add 1 for the background class
    # Load pretrained HoVer-Net
    model = hovernet.HoverNet.from_pretrained("hgsc_v1_efficientnet_b5", device=torch.device("cpu"))


    if model.model.heads["type"]["nuc_type"] != num_classes:
        model.model.heads["type"]["nuc_type"] = num_classes  # update the config
        model.model.decoder.heads["type"]["nuc_type"] = torch.nn.Conv2d(64, num_classes, kernel_size=1)
        print(f"🔧 Updated 'nuc_type' head to {num_classes} classes.")
        
    model.model.to(DEVICE)


    # --- Setup Data ---
    transform = Compose([
        Resize(CONFIG.IMG_SIZE, CONFIG.IMG_SIZE),
        Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ToTensorV2()
    ])
    
    train_dataset = NucleiDataset(CONFIG.TRAIN_IMG_DIR, CONFIG.TRAIN_XML_DIR, transform=transform)
    val_dataset = NucleiDataset(CONFIG.VAL_IMG_DIR, CONFIG.VAL_XML_DIR, transform=transform)
    
    train_loader = DataLoader(train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False, num_workers=0)

    # --- Optimizer and Training Loop ---
    optimizer = optim.Adam(model.model.parameters(), lr=CONFIG.LR)
    best_val_loss = float('inf')

    print("--- Starting Training ---")
    for epoch in range(CONFIG.EPOCHS):
        model.model.train()
        total_train_loss = 0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{CONFIG.EPOCHS} [Train]"):
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            
            # The model computes its own loss when labels are provided
            images = images.to(DEVICE)
            inst_maps = labels[:, 0, :, :].to(DEVICE)  # instance map
            type_maps = labels[:, 1, :, :].to(DEVICE)  # type map

            outputs = model.model(images)  # forward pass
            pred_inst_maps = outputs.nuc.nuc_map      # predicted instance maps
            pred_type_maps = outputs.type.nuc_type     # predicted type maps

            # --- Compute losses manually ---
            # Use CrossEntropy for type maps
            criterion = torch.nn.CrossEntropyLoss()
            loss_type = criterion(pred_type_maps, type_maps)

            # For nuclei map, you can implement dice loss / mse / hover loss if available
            loss_inst = torch.tensor(0.0, device=DEVICE)  # placeholder if you don’t have hover loss

            loss = loss_type + loss_inst
            
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        # --- Validation Step ---
        model.model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{CONFIG.EPOCHS} [Val]"):
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                images = images.to(DEVICE)
                inst_maps = labels[:, 0, :, :].to(DEVICE)  # instance map
                type_maps = labels[:, 1, :, :].to(DEVICE)  # type map

                outputs = model.model(images)  # forward pass
                pred_inst_maps = outputs.nuc.nuc_map       # predicted instance maps
                pred_type_maps = outputs.type.nuc_type     # predicted type maps

                # --- Compute losses manually ---
                # Use CrossEntropy for type maps
                criterion = torch.nn.CrossEntropyLoss()
                loss_type = criterion(pred_type_maps, type_maps)

                # For nuclei map, you can implement dice loss / mse / hover loss if available
                loss_inst = torch.tensor(0.0, device=DEVICE)  # placeholder if you don’t have hover loss

                loss = loss_type + loss_inst
                total_val_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = total_val_loss / len(val_loader)
        
        print(f"Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), CONFIG.MODEL_SAVE_PATH)
            print(f"✅ New best model saved with validation loss: {best_val_loss:.4f}")

if not os.path.exists(CONFIG.MODEL_SAVE_PATH):
    train_and_validate()
else:
    print(f"Model already trained. Found at: {CONFIG.MODEL_SAVE_PATH}")

# %% [markdown]
# ## 4. Inference and Submission
#
# We load our best-performing model and run it on the test set to generate the final `submission.csv` file.

# %%
def run_inference():
    print("\n--- Running Full Inference Pipeline on Test Set ---")
    
    # --- CORRECTED Model Loading ---
    num_classes = len(CONFIG.CLASSES) + 1
    model = hovernet.HoverNet.from_pretrained("hgsc_v1_efficientnet_b5", device=torch.device("cpu"))
    # If your dataset has a different number of nucleus types, adjust the output head
    if model.model.heads["type"]["nuc_type"] != num_classes:
        model.model.heads["type"]["nuc_type"] = num_classes  # update the config
        model.model.decoder.heads["type"]["nuc_type"] = torch.nn.Conv2d(64, num_classes, kernel_size=1)
        print(f"🔧 Updated 'nuc_type' head to {num_classes} classes.")
    model.load_state_dict(torch.load(CONFIG.MODEL_SAVE_PATH, map_location=DEVICE))
    model.to(DEVICE)
    model.set_inference_mode()

    # --- Inference Transforms ---
    transform = Compose([
        Resize(CONFIG.IMG_SIZE, CONFIG.IMG_SIZE),
        Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ToTensorV2()
    ])
    
    submission_data = []
    test_files = [f for f in os.listdir(CONFIG.TEST_IMG_DIR) if f.endswith(".tif")]

    for filename in tqdm(test_files, desc="Generating predictions"):
        image_id = os.path.splitext(filename)[0]
        img_path = os.path.join(CONFIG.TEST_IMG_DIR, filename)
        
        image_orig = cv2.imread(img_path)
        image_orig = cv2.cvtColor(image_orig, cv2.COLOR_BGR2RGB)
        h_orig, w_orig, _ = image_orig.shape

        # Transform image for model input
        image_tensor = transform(image=image_orig)["image"].unsqueeze(0).to(DEVICE)

        with torch.no_grad():
            # Get predictions
            prob_map = model.predict(image_tensor)
            output = model.post_process(prob_map)["nuc"][0]
            pred_inst_map, pred_type_map = output[0], output[1]
            
            # Resize back to original image size
            pred_inst_map = cv2.resize(pred_inst_map.astype(np.uint16), (w_orig, h_orig), interpolation=cv2.INTER_NEAREST)
            pred_type_map = cv2.resize(pred_type_map.astype(np.uint8), (w_orig, h_orig), interpolation=cv2.INTER_NEAREST)

        # Convert model output to the required submission format
        final_masks = {cls: np.zeros((h_orig, w_orig), dtype=np.int32) for cls in CONFIG.CLASSES}
        
        for inst_id in range(1, pred_inst_map.max() + 1):
            inst_mask = (pred_inst_map == inst_id)
            if inst_mask.sum() == 0: continue
            
            inst_type = np.median(pred_type_map[inst_mask]).astype(int)
            
            if inst_type in CONFIG.INV_CLASS_MAP:
                class_name = CONFIG.INV_CLASS_MAP[inst_type]
                final_masks[class_name][inst_mask] = inst_id

        row = {"image_id": image_id}
        for class_name in ["Epithelial", "Lymphocyte", "Neutrophil", "Macrophage"]:
            row[class_name] = rle_encode_instance_mask(final_masks[class_name])
        submission_data.append(row)

    # --- Create Submission File ---
    submission_df = pd.DataFrame(submission_data)
    submission_df = submission_df[["image_id", "Epithelial", "Lymphocyte", "Neutrophil", "Macrophage"]]
    submission_df.to_csv("submission.csv", index=False)

    print("\nSubmission file 'submission.csv' created successfully!")
    return submission_df

submission_df = run_inference()
print("\n--- Final Submission Preview ---")
print(submission_df.head())

Using Apple MPS (GPU) 🚀
--- Initializing Model and DataLoaders ---
🔧 Updated 'nuc_type' head to 5 classes.
--- Starting Training ---


Epoch 1/30 [Train]:   0%|          | 0/53 [00:00<?, ?it/s]


AttributeError: 'dict' object has no attribute 'nuc'